In [ ]:
import requests
import csv
import time

In [ ]:
checkI = 0 

In [ ]:
def storeInfo(as_dict_vulnerability):
    with open('2021.csv', 'a',  newline='') as file:
        writer = csv.writer(file)
        writer.writerow([as_dict_vulnerability['vulnerability_id'], as_dict_vulnerability['reporting_time'],as_dict_vulnerability['vulnerability_description'],
                       as_dict_vulnerability['references'],as_dict_vulnerability['CWE-ID'],as_dict_vulnerability['CWE-Name'],as_dict_vulnerability['CVSS-Score']])

In [ ]:
def getVulInfo(urls, urlll):
    global checkI
    as_dict_vulnerability = {
    "vulnerability_id": None, 
    "reporting_time": None, 
    "vulnerability_description": None,
    "references": None,
    "CWE-ID": None, 
    "CWE-Name": None,
    "CVSS-Score": None,
    }
    for i in range(checkI, len(urls)):
        web = requests.get(urls[i], allow_redirects=True)
        raw_data = web.content.decode("UTF-8")
        
        #get id
        id = raw_data.split("<title>NVD - ")
        id = id[1].split("<")[0]
        print(id)
        
        #get date
        date = raw_data.split("vuln-published-on\">")
        date = date[1].split("<")[0]

        #get reference
        reference = raw_data.split("vuln-hyperlinks-row-")
        reference.pop(0)

        links = ""
        n = 2

        for k in reference:
            n -= 1
            href = k.split("href=\"")[1]
            href = href.split("\"")[0]
            links += href
            if (n):
                links += ","

        #get CWE id and name
        try:
            CWE_id = raw_data.split("vuln-CWEs-link-0\">")[1]
            CWE_name = raw_data.split("vuln-CWEs-link-0\">")[2].split("<")[0]

            if (len(CWE_id.split("href=\"")) > 1):
                CWE_id = CWE_id.split("href=\"")[1].split("\"")[0]
            else:
                CWE_id = CWE_id.split("<span>")[1].split("<")[0]
        except Exception as e: 
            CWE_id = "Unavailable"
            CWE_name = "Unavailable"

        #get CVSS_score
        CVSS_score = raw_data.split("vuln-cvss3-panel-score")
            
        if (len(CVSS_score) > 1):
            CVSS_score = raw_data.split("vuln-cvss3-panel-score")[1]
            CVSS_score = CVSS_score.split(">")[1].split(" ")[0]
        else:
            CVSS_score = "N/A"
        print(CVSS_score)
        #get description
        description = raw_data.split("description\">")
        description = description[1].split("<")[0]
        
         #fill dictionary
        as_dict_vulnerability.update({"vulnerability_id": id})
        as_dict_vulnerability.update({"reporting_time": date})
        as_dict_vulnerability.update({"vulnerability_description": description})
        as_dict_vulnerability.update({"references": links})
        as_dict_vulnerability.update({"CWE-ID": CWE_id})
        as_dict_vulnerability.update({"CWE-Name": CWE_name})
        as_dict_vulnerability.update({"CVSS-Score": CVSS_score})
        storeInfo(as_dict_vulnerability)
        
        if (checkI % 50 == 0 and checkI > 0):
            print(checkI)
        checkI += 1
        
    checkI = 0

In [ ]:
def getVulnerabilities():
    Year = 2021
    dict_arr_vulnerability = []
    global checkI
    checkj = 0
    j = 1

    while j <= 12:
        try:
            if checkj != j:
                checkj = j
                urlll = 'https://nvd.nist.gov/vuln/full-listing/{year}/{tag}'
                urlll = urlll.format(year = Year, tag = j)
                print(urlll)
                r = requests.get(urlll, allow_redirects=True)
                raw = r.content.decode("UTF-8")
                step1 = raw.split("/vuln/detail/CVE-") #list of CVE links eg: ("<a href="/vuln/detail/CVE-2012-10001">CVE-2012-10001</a>")
                step2 = [] #list of CVE ids
                for i in step1:
                    step2.append(i.split("\">")[0]) #append just CVE id into step2

                step2.pop(0) #remove <html lang="en
                urls = []

                #create CVE urls
                for i in step2:
                    url = "https://nvd.nist.gov/vuln/detail/CVE-{tag}"
                    url = url.format(tag = i)
                    urls.append(url)
                print("Number of vulnerabilities left: %d"%(len(urls) - checkI))
                getVulInfo(urls, urlll)
                    
        except Exception as e: 
            print("An Error has Occured")
            print(e)
#             j -= 1
#             print("\nFailed at checkI: %d"%(checkI))
#             print("\nSkipped url: " + urls[checkI] + "\n")
#             checkI += 1

#             print("Program will sleep for 10 seconds")
#             time.sleep(10)
#             print("Program is operational again")

        j += 1
        
    
    print("Program Complete")

In [ ]:
getVulnerabilities()

In [ ]:
def analyzeData(fileName):
    count = 0
    with open(fileName, mode='r', encoding='utf-8-sig') as f:
        vuls = csv.DictReader(f)
        for row in vuls:
            if "** REJECT **" in row['vulnerability_description']:
                if "** REJECT **  DO NOT USE THIS CANDIDATE NUMBER." in row['vulnerability_description']:
                    print(row['vulnerability_description'])
#                     "** REJECT ** DO NOT USE THIS CANDIDATE NUMBER"
                count+=1
    print(count)
    print(132270-count)

In [ ]:
analyzeData("NVD_2011-2021.csv")